In [1]:
from stable_baselines3 import DQN
from stable_baselines3.dqn import MlpPolicy
from stable_baselines3 import PPO
from typing import TYPE_CHECKING, Any, Callable, Dict, List, Optional, Union
from stable_baselines3.common.callbacks import EvalCallback, BaseCallback
from stable_baselines3.common.vec_env import DummyVecEnv, VecEnv, sync_envs_normalization
from stable_baselines3.common.monitor import Monitor
import gymnasium as gym

In [2]:
from gymnasium_env.envs.f1_env import RewardFunction
from gymnasium_env.envs.f1_env import RewardFunctionPerPositionAtFinalLap

from racesim.config import GridConfigMixed
from racesim.config import GridConfig

In [3]:
class F1EnvEvalCallback(EvalCallback):
    def __init__(
        self,
        eval_env: Union[gym.Env, VecEnv],
        callback_on_new_best: Optional[BaseCallback] = None,
        callback_after_eval: Optional[BaseCallback] = None,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        log_path: Optional[str] = None,
        best_model_save_path: Optional[str] = None,
        deterministic: bool = True,
        render: bool = False,
        verbose: int = 1,
        warn: bool = True,
        seed_eval_env: int = 0,
        reward_function: RewardFunction = None,
        grid_config: GridConfig = None
    ):
        super().__init__(eval_env,callback_on_new_best,callback_after_eval,n_eval_episodes,eval_freq,
                         log_path,best_model_save_path,deterministic,render,verbose,warn)
        self.seed_eval_env = seed_eval_env
        self.reward_function = reward_function
        self.grid_config = grid_config

    def _on_step(self) -> bool:
        continue_training = super()._on_step()
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            self.set_env_for_next_eval()
        return continue_training
        

    def set_env_for_next_eval(self) -> None:
        self.eval_env = gym.make("F1Env/Basic-v0", seed=self.seed_eval_env,
                                 reward_function=self.reward_function, grid_config=self.grid_config)

In [4]:
seed = 0
n_seeds = 3
seed_eval = 50
reward_function = RewardFunctionPerPositionAtFinalLap()

grid_config = GridConfigMixed()

In [5]:
for i in range(n_seeds):
    print("-----------------\n\n" + " RUN " + str(i) + "\n\n--------------------------")
    env = gym.make("F1Env/Basic-v0", seed=i, reward_function=reward_function, grid_config=grid_config)
    env_to_eval = gym.make("F1Env/Basic-v0", seed=seed_eval, reward_function=reward_function, grid_config=grid_config)

    eval_callback = F1EnvEvalCallback(eval_env=env_to_eval, best_model_save_path="./logs_" + str(i) + "/",
                             log_path="./logs_" + str(i) + "/", n_eval_episodes=20, eval_freq=25000,
                             deterministic=True, render=False, seed_eval_env=seed_eval,
                                      reward_function=reward_function, grid_config=grid_config)

    model_reward_position_change = DQN("MlpPolicy", env, gamma=1, buffer_size=100000, learning_starts=5000,
                                            train_freq=(16,"step"), exploration_initial_eps=0.25, exploration_fraction=0.2,
                                            exploration_final_eps=0.020, learning_rate=0.00005,
                                            batch_size=256,
                                            verbose=0, tensorboard_log="./dqn_r3/", seed=i)
    model_reward_position_change.learn(total_timesteps=1e6, callback=eval_callback)
    model_reward_position_change.save(path="./model_reward_position_change_"+str(i))

-----------------

 RUN 0

--------------------------


/home/usuario/miniconda3/envs/rfl/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=25000, episode_reward=-1.00 +/- 0.00
Episode length: 77.00 +/- 0.00
New best mean reward!
Eval num_timesteps=50000, episode_reward=-0.40 +/- 0.49
Episode length: 77.00 +/- 0.00
New best mean reward!
Eval num_timesteps=75000, episode_reward=-0.50 +/- 0.50
Episode length: 77.00 +/- 0.00
Eval num_timesteps=100000, episode_reward=0.41 +/- 2.47
Episode length: 77.00 +/- 0.00
New best mean reward!
Eval num_timesteps=125000, episode_reward=-0.03 +/- 1.02
Episode length: 77.00 +/- 0.00
Eval num_timesteps=150000, episode_reward=0.55 +/- 1.66
Episode length: 77.00 +/- 0.00
New best mean reward!
Eval num_timesteps=175000, episode_reward=0.86 +/- 1.86
Episode length: 77.00 +/- 0.00
New best mean reward!
Eval num_timesteps=200000, episode_reward=0.49 +/- 1.65
Episode length: 77.00 +/- 0.00
Eval num_timesteps=225000, episode_reward=1.71 +/- 1.81
Episode length: 77.00 +/- 0.00
New best mean reward!
Eval num_timesteps=250000, episode_reward=0.60 +/- 1.41
Episode length: 77.00 +/- 0.